In [1]:
import pandas as pd
import numpy as np

data = "/mnt/d/Data/mangaki-data-challenge/"

record = pd.read_csv(data+'watched.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16,
    'rating': 'category'
})

train_full = pd.read_csv(data+'train_withcv.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16,
    'rating': np.int8,
    'cv': np.int8
})

test = pd.read_csv(data+'test.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16
})

In [2]:
u1 = pd.crosstab(record.user_id, record.rating).add_prefix('user_').apply(lambda r: r/r.sum(), axis=1)

In [3]:
ratelut = {
    'dislike':1.0,
    'like':3.0,
    'love':4.0,
    'neutral':2.0
}
record['score']=record.rating.map(lambda x: ratelut[x]).astype(float)
u2 = record[['user_id', 'score']].groupby(by='user_id')['score'].agg(['mean', 'std']).rename(columns={'mean':'user_mean', 'std':'user_std'})

In [4]:
i1 = pd.crosstab(record.work_id, record.rating).add_prefix('work_').apply(lambda r: r/r.sum(), axis=1)
i2 = record[['work_id', 'score']].groupby(by='work_id')['score'].agg(['mean', 'std']).add_prefix('item_')

In [7]:
# deprecated
valid = train_full[train_full.cv==3]
train = train_full[~train_full.cv.isin([3])]
u3 = pd.crosstab(train.user_id, train.rating).add_prefix('user_').apply(lambda r: r/r.sum(), axis=1)
i3 = pd.crosstab(train.work_id, train.rating).add_prefix('item_').apply(lambda r: r/r.sum(), axis=1)

In [9]:
valid.shape

(3754, 4)

In [10]:
for cv in [1,2,3]:
    valid = train_full[train_full.cv==cv]
    train = train_full[~train_full.cv.isin([cv])]
    train = train.merge(u1, left_on='user_id', right_index=True, how='left').\
                  merge(u2, left_on='user_id', right_index=True, how='left').\
                  merge(i1, left_on='work_id', right_index=True, how='left').\
                  merge(i2, left_on='work_id', right_index=True, how='left')
    valid = valid.merge(u1, left_on='user_id', right_index=True, how='left').\
                  merge(u2, left_on='user_id', right_index=True, how='left').\
                  merge(i1, left_on='work_id', right_index=True, how='left').\
                  merge(i2, left_on='work_id', right_index=True, how='left')
    train = train.drop(['cv'], axis=1)
    train.to_csv(data+"train_{0}.csv".format(cv), index=False)
    valid = valid.drop(['cv'], axis=1)
    valid.to_csv(data+"valid_{0}.csv".format(cv), index=False)

In [27]:
train = train.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left')

In [28]:
train = train.drop(['cv'], axis=1)
train.to_csv(data+"train_1.csv", index=False)

In [20]:
valid = valid.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left')
valid = valid.drop(['cv'], axis=1)

In [21]:
valid.to_csv(data+"valid_3.csv", index=False)

In [11]:
train = train_full
#u3 = pd.crosstab(train.user_id, train.rating).add_prefix('user_').apply(lambda r: r/r.sum(), axis=1)
#i3 = pd.crosstab(train.work_id, train.rating).add_prefix('item_').apply(lambda r: r/r.sum(), axis=1)
train = train.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left')
train = train.drop(['cv'], axis=1)
train.to_csv(data+"baseline/train_0.csv", index=False)

In [12]:
test = test.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left')
test.to_csv(data+"baseline/test_0.csv", index=False)

W2V feature

In [22]:
w2vdim=10

In [23]:
from gensim.models.word2vec import Word2Vec
from random import sample

record_positive = record[record.rating.isin(['like', 'love'])]
record_positive_effectiveuser = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_positive_effectiveuser = record_positive_effectiveuser[record_positive_effectiveuser.work_count>5]
record_positive = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
record_positive = pd.concat((record_positive, record_positive_effectiveuser), axis=1, join='inner')

sentences = []
for itm in record_positive.iterrows():
    t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

item_w2v_positive = []
item_index = []
for item in range(9897):
    if 'i{0}'.format(item) in w2v_positive:
        item_index.append(item)
        item_w2v_positive.append(w2v_positive['i{0}'.format(item)])
item_w2v_positive = np.vstack(item_w2v_positive)
i4 = pd.DataFrame(data = item_w2v_positive, index=item_index, columns=['itemw2vpos_{0}'.format(i) for i in range(w2vdim)])
i4.index.name='work_id'

i4.reset_index().to_csv(data+"features/item_w2v_shuffled_{0}d.csv".format(w2vdim), index=False)

In [24]:
# for user
record_positive = record[record.rating.isin(['like', 'love'])]
record_positive_effectiveitem = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_positive_effectiveitem = record_positive_effectiveitem[record_positive_effectiveitem.user_count>5]
record_positive = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
record_positive = pd.concat((record_positive, record_positive_effectiveitem), axis=1, join='inner')

sentences = []
for itm in record_positive.iterrows():
    t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

user_w2v_positive = []
user_index = []
for user in range(1983):
    if 'u{0}'.format(user) in w2v_positive:
        user_index.append(user)
        user_w2v_positive.append(w2v_positive['u{0}'.format(user)])
user_w2v_positive = np.vstack(user_w2v_positive)
u4 = pd.DataFrame(data = user_w2v_positive, index=user_index, columns=['userw2vpos_{0}'.format(i) for i in range(w2vdim)])
u4.index.name='user_id'

u4.reset_index().to_csv(data+"features/user_w2v_shuffled_{0}d.csv".format(w2vdim), index=False)

In [25]:
from gensim.models.word2vec import Word2Vec

record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveuser = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_negative_effectiveuser = record_negative_effectiveuser[record_negative_effectiveuser.work_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveuser), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_negative = Word2Vec(sentences, size=w2vdim)

item_w2v_negative = []
item_index = []
for item in range(9897):
    if 'i{0}'.format(item) in w2v_negative:
        item_index.append(item)
        item_w2v_negative.append(w2v_negative['i{0}'.format(item)])
item_w2v_negative = np.vstack(item_w2v_negative)
i4 = pd.DataFrame(data = item_w2v_negative, index=item_index, columns=['itemw2vpos_{0}'.format(i) for i in range(w2vdim)])
i4.index.name='work_id'

i4.reset_index().to_csv(data+"features/item_w2vneg_shuffled_{0}d.csv".format(w2vdim), index=False)

In [26]:
record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveitem = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_negative_effectiveitem = record_negative_effectiveitem[record_negative_effectiveitem.user_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveitem), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_negative = Word2Vec(sentences, size=w2vdim)

user_w2v_negative = []
user_index = []
for user in range(1983):
    if 'u{0}'.format(user) in w2v_negative:
        user_index.append(user)
        user_w2v_negative.append(w2v_negative['u{0}'.format(user)])
user_w2v_negative = np.vstack(user_w2v_negative)
u4 = pd.DataFrame(data = user_w2v_negative, index=user_index, columns=['userw2vpos_{0}'.format(i) for i in range(w2vdim)])
u4.index.name='user_id'

u4.reset_index().to_csv(data+"features/user_w2vneg_shuffled_{0}d.csv".format(w2vdim), index=False)

LDA feature

In [25]:
num_topics = 10

In [26]:
from gensim.models.ldamodel import LdaModel

record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive_effectiveuser = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_positive_effectiveuser = record_positive_effectiveuser[record_positive_effectiveuser.work_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveuser), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5}
for itm in record_positive.iterrows():
    for work in itm[1].work_id:
        strwork = 'i{0}'.format(work)
        if strwork not in dictionary:
            dictionary[strwork]=cnt
            cnt+=1
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lda_user = LdaModel(corpus=corpus, num_topics=num_topics, id2word=invdict)

record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating) if 'i{0}'.format(i) in dictionary])

user_lda_positive = []
for i, topic in enumerate(lda_user[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    user_lda_positive.append(temp)
user_lda_positive = np.vstack(user_lda_positive)
u5 = pd.DataFrame(data = user_lda_positive, index=record_positive.index, columns=['user_ldapos_{0}'.format(i) for i in range(num_topics)])

u5.reset_index().to_csv(data+"features/user_lda_{0}d.csv".format(num_topics), index=False)

In [27]:
record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive_effectiveitem = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_positive_effectiveitem = record_positive_effectiveitem[record_positive_effectiveitem.user_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveitem), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5}
for itm in record_positive.iterrows():
    for user in itm[1].user_id:
        struser = 'u{0}'.format(user)
        if struser not in dictionary:
            dictionary[struser]=cnt
            cnt+=1
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lda_item = LdaModel(corpus=corpus, num_topics=num_topics, id2word=invdict)

record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating) if 'u{0}'.format(i) in dictionary])

item_lda_positive = []
for i, topic in enumerate(lda_item[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    item_lda_positive.append(temp)
item_lda_positive = np.vstack(item_lda_positive)
i5 = pd.DataFrame(data = item_lda_positive, index=record_positive.index, columns=['item_ldapos_{0}'.format(i) for i in range(num_topics)])

i5.reset_index().to_csv(data+"features/item_lda_{0}d.csv".format(num_topics), index=False)

In [31]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveuser = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_negative_effectiveuser = record_negative_effectiveuser[record_negative_effectiveuser.work_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveuser), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['i{0}'.format(str(i)) for i in itm[1].work_list])
    
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(rc) for rc in sentences]

lda_user = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

record_negative = record[record.rating.isin(['dislike'])]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['i{0}'.format(str(i)) for i in itm[1].work_list])
corpus = [dictionary.doc2bow(rc) for rc in sentences]

user_lda_negative = []
for i, topic in enumerate(lda_user[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    user_lda_negative.append(temp)
user_lda_negative = np.vstack(user_lda_negative)
u5 = pd.DataFrame(data = user_lda_negative, index=record_negative.index, columns=['user_ldapos_{0}'.format(i) for i in range(num_topics)])

u5.reset_index().to_csv(data+"features/user_ldaneg_{0}d.csv".format(num_topics), index=False)

In [32]:
record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveitem = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_negative_effectiveitem = record_negative_effectiveitem[record_negative_effectiveitem.user_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveitem), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['u{0}'.format(str(i)) for i in itm[1].user_list])
    
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(rc) for rc in sentences]

lda_item = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

record_negative = record[record.rating.isin(['like', 'love'])]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['u{0}'.format(str(i)) for i in itm[1].user_list])
corpus = [dictionary.doc2bow(rc) for rc in sentences]

item_lda_negative = []
for i, topic in enumerate(lda_item[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    item_lda_negative.append(temp)
item_lda_negative = np.vstack(item_lda_negative)
i5 = pd.DataFrame(data = item_lda_negative, index=record_negative.index, columns=['item_ldapos_{0}'.format(i) for i in range(num_topics)])

i5.reset_index().to_csv(data+"features/item_ldaneg_{0}d.csv".format(num_topics), index=False)

convert to libfm format

In [55]:
from scipy.sparse import coo_matrix, hstack
from sklearn.datasets import dump_svmlight_file


path = "/mnt/d/Data/mangaki-data-challenge/"
for cv in [1,2,3]:
    train = pd.read_csv(path+"baseline/train_{0}.csv".format(cv))
    valid = pd.read_csv(path+"baseline/valid_{0}.csv".format(cv))
    
    data = []
    row = []
    col = []
    for i, itm in enumerate(train.iterrows()):
        row.append(i)
        col.append(itm[1].user_id)
        data.append(1.0)
        row.append(i)
        col.append(itm[1].work_id+1983)
        data.append(1.0)

    basic = coo_matrix((data, (row, col)), shape=(train.shape[0], 1983+9897))
    ext = train.ix[:, 3:].fillna(0).values
    ext[:, 4] = ext[:, 4]/4
    ext[:, 10] = ext[:, 10]/4
    value = hstack([basic, ext])
    dump_svmlight_file(value, train.rating.values, path+"libfm/train_{0}.csv".format(cv))
    
    data = []
    row = []
    col = []
    for i, itm in enumerate(valid.iterrows()):
        row.append(i)
        col.append(itm[1].user_id)
        data.append(1.0)
        row.append(i)
        col.append(itm[1].work_id+1983)
        data.append(1.0)

    basic = coo_matrix((data, (row, col)), shape=(valid.shape[0], 1983+9897))
    ext = valid.ix[:, 3:].fillna(0).values
    ext[:, 4] = ext[:, 4]/4
    ext[:, 10] = ext[:, 10]/4
    value = hstack([basic, ext])
    dump_svmlight_file(value, valid.rating.values, path+"libfm/valid_{0}.csv".format(cv))

In [58]:
train = pd.read_csv(path+"baseline/train_0.csv")
valid = pd.read_csv(path+"baseline/test_0.csv")

data = []
row = []
col = []
for i, itm in enumerate(train.iterrows()):
    row.append(i)
    col.append(itm[1].user_id)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].work_id+1983)
    data.append(1.0)

basic = coo_matrix((data, (row, col)), shape=(train.shape[0], 1983+9897))
ext = train.ix[:, 3:].fillna(0).values
ext[:, 4] = ext[:, 4]/4
ext[:, 10] = ext[:, 10]/4
value = hstack([basic, ext])
dump_svmlight_file(value, train.rating.values, path+"libfm/train_0.csv")

data = []
row = []
col = []
for i, itm in enumerate(valid.iterrows()):
    row.append(i)
    col.append(itm[1].user_id)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].work_id+1983)
    data.append(1.0)

basic = coo_matrix((data, (row, col)), shape=(valid.shape[0], 1983+9897))
ext = valid.ix[:, 3:].fillna(0).values
ext[:, 4] = ext[:, 4]/4
ext[:, 10] = ext[:, 10]/4
value = hstack([basic, ext])
dump_svmlight_file(value, np.zeros(valid.shape[0]), path+"libfm/test_0.csv")